In [1]:
import pandas as pd
from tqdm import tqdm

## Preprocess ratings from txt to csv while keeping only essential information

In [2]:
# RateBeer (<60s on my computer, around 22Gb of ram)
ratings_list = []
rating_dic = {}
with open("data/RateBeer/ratings.txt") as f:
    for i, line in tqdm(enumerate(f)):
        field = line.split(": ")[0]
        if field == "\n":
            ratings_list.append(rating_dic)
            rating_dic = {}
            continue
        content = line.split(": ")[1:]
        content = ": ".join(content)
        rating_dic[field] = content.strip()
            
ratings_RB = pd.DataFrame.from_dict(ratings_list)
ratings_RB.to_csv("data/RateBeer/ratings.csv", index=False)

121075258it [00:50, 2390794.40it/s]


In [5]:
del ratings_RB, ratings_list

NameError: name 'ratings_RB' is not defined

In [6]:
# BeerAdvocate (<60s on my computer, around 28Gb of ram)
ratings_list = []
rating_dic = {}
with open("data/BeerAdvocate/ratings.txt") as f:
    for i, line in tqdm(enumerate(f)):
        field = line.split(": ")[0]
        if field == "\n":
            ratings_list.append(rating_dic)
            rating_dic = {}
            continue
        content = line.split(": ")[1:]
        content = ": ".join(content)
        rating_dic[field] = content.strip()
            
ratings_BA = pd.DataFrame.from_dict(ratings_list)
ratings_BA.to_csv("data/BeerAdvocate/ratings.csv", index=False)

151074576it [00:57, 2633671.21it/s]


In [7]:
del ratings_BA, ratings_list

## Load data

In [ ]:
# Data from RateBeer 'RB'
beers_RB = pd.read_table("./data/RateBeer/beers.csv", sep=",")
breweries_RB = pd.read_table("./data/RateBeer/breweries.csv", sep=",").rename(columns={"id":"brewery_id","location":"brewery_location","name":"brewery_name"})
users_RB = pd.read_table("./data/RateBeer/users.csv", sep=",").rename(columns={"location":"user_location"})
ratings_RB = pd.read_table("./data/RateBeer/ratings.csv", sep=",")

# Data from BeerAdvocate 'BA'
beers_BA = pd.read_table("./data/BeerAdvocate/beers.csv", sep=",")
breweries_BA = pd.read_table("./data/BeerAdvocate/breweries.csv", sep=",").rename(columns={"id":"brewery_id","location":"brewery_location","name":"brewery_name"})
users_BA = pd.read_table("./data/BeerAdvocate/users.csv", sep=",").rename(columns={"location":"user_location"})
ratings_BA = pd.read_table("./data/BeerAdvocate/ratings.csv", sep=",")

# Data from MixedDataset 'MD'
beers_MD = pd.read_table("./data/matched_beaer_data/beers.csv", sep=",")
breweries_MD = pd.read_table("./data/matched_beer_data/breweries.csv", sep=",")
users_MD = pd.read_table("./data/matched_beer_data/users.csv", sep=",")
users_approx_MD = pd.read_table("./data/matched_beer_data/users_approx.csv", sep=",")
ratings_MD = pd.read_table("./data/matched_beer_data/ratings.csv", sep=",")

## Merge data by dataset (every information available populates ratings)

In [ ]:
# Data from RateBeer 'RB'
ratings_RB = ratings_RB.merge(users_RB ,on="user_id", how="left", suffixes=('', '_drop'))
ratings_RBm = ratings_RBm.merge(breweries_RB,on="brewery_id", how="left", suffixes=('', '_drop'))
ratings_RBm = ratings_RBm.merge(beers_RB,on="beer_id", how="left", suffixes=('', '_drop'))
ratings_RBm.drop([col for col in ratings_RBm.columns if 'drop' in col], axis=1, inplace=True)
ratings_RBm['dataset']="RB"

# Data from BeerAdvocate 'BA'
ratings_BAm = ratings_BA.merge(users_BA ,on="user_id", how="left", suffixes=('', '_drop'))
ratings_BAm = ratings_BAm.merge(breweries_BA,on="brewery_id", how="left", suffixes=('', '_drop'))
ratings_BAm = ratings_BAm.merge(beers_BA,on="beer_id", how="left", suffixes=('', '_drop'))
ratings_BAm.drop([col for col in ratings_BAm.columns if 'drop' in col], axis=1, inplace=True)
ratings_BAm['dataset']="BA"

## Merge ratings_BAm and ratings_RBm

In [ ]:
ratings_mixed = pd.concat([ratings_BAm, ratings_RBm], axis=0, ignore_index=True)
ratings_mixed.to_csv("data/ratings.csv", index=False)